In [1]:
import pandas as pd
import string
import re
import nltk
from nltk.corpus import stopwords
import random
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
DATAFILE_model = '/Users/rodrigo/Documents/Big-Data/final-project/hhs_ca/ML/df_model.csv'

df = pd.read_csv(DATAFILE_model, low_memory = False)
df = df[['total_obligation', 'transaction_description', 'business_categories', 'recipient_location_county_code']]
df = df[pd.notnull(df['transaction_description'])]
print(df.shape)
df.tail()

(207892, 4)


,total_obligation,transaction_description,business_categories,recipient_location_county_code
258358,1.558053e+09,MEDICARE: HOSPITAL INSURANCE,NaN,37
258359,1.576352e+07,MEDICARE: SUPPLEMENTARY MEDICAL INSURANCE,NaN,67
258361,2.803770e+06,MEDICARE: SUPPLEMENTARY MEDICAL INSURANCE,NaN,73
258362,-2.928280e+05,MEDICARE: SUPPLEMENTARY MEDICAL INSURANCE,other_than_small_business category_business,111
258363,1.283526e+07,MEDICARE: SUPPLEMENTARY MEDICAL INSURANCE,NaN,1


### variation: delete some classes

In [34]:

df = df[df.recipient_location_county_code != 51]
df = df[df.recipient_location_county_code != 3]
df = df[df.recipient_location_county_code != 43]
#& df.recipient_location_county_code != 53 & df.recipient_location_county_code != 43]
df.shape

(207883, 4)

In [3]:
DATAFILE_na = '/Users/rodrigo/Documents/Big-Data/final-project/hhs_ca/ML/df_NA_2015.csv'
df_na = pd.read_csv(DATAFILE_na, low_memory = False)
print(df_na.shape)
df_na = df_na[['total_obligation', 'transaction_description', 'business_categories', 'recipient_location_county_code']]
df_na = df_na[pd.notnull(df_na['transaction_description'])]
df_na.tail()

(1552, 62)


,total_obligation,transaction_description,business_categories,recipient_location_county_code
1547,20730.69,IGF::CT::IGF OT LODGING AND MEETING ROOM PACKA...,other_than_small_business partnership_or_limit...,NaN
1548,21501.39,IGF::CT::IGF OT LODGING AND MEETING ROOM PACKA...,other_than_small_business partnership_or_limit...,NaN
1549,18007.58,IGF::CT::IGF OT LODGING AND MEETING ROOM PACKA...,other_than_small_business partnership_or_limit...,NaN
1550,21904.88,IGF::CT::IGF OT LODGING AND MEETING ROOM PACKA...,other_than_small_business partnership_or_limit...,NaN
1551,18007.58,IGF::CT::IGF OT LODGING AND MEETING ROOM PACKA...,other_than_small_business partnership_or_limit...,NaN


In [100]:
counties_names = {'Alameda ' : 1 , 'Alpine ' : 3 , 'Amador' : 5, 'Butte' : 7 , 'Calaveras' : 9 , 'Colusa' : 11  , 'Contra Costa' : 13 , 
'Del Norte' : 15 , 'El Dorado' : 17 , 'Fresno' : 19 , 'Glenn' : 21, 'Humboldt' : 23 , 'Imperial' : 25 ,'Inyo' : 27 , 
 'Kern' : 29 , 'Kings' : 31, 'Lake' : 33 , 'Lassen' : 35 , 'Los Angeles' : 37 , 'Madera' : 39 , 'Marin' : 41 , 
 'Mariposa' : 43 , 'Mendocino' : 45 , 'Merced'  : 47 , 'Modoc' : 49 , 'Mono' : 51 ,  'Monterey' : 53 , 'Napa' : 55 , 
 'Nevada' : 57 , 'Orange' : 59 , 'Placer' : 61 , 'Plumas' : 63 , 'Riverside' : 65 , 'Sacramento' : 67 , 'San Benito' : 69 ,
 'San Bernardino' : 71 , 'San Diego' : 73 , 'San Francisco' : 75, 'San Joaquin ' : 77 , 'San Luis Obispo ' : 79 , 
 'San Mate' : 81 , 'Santa Barbara' : 83, 'Santa Clara' : 85 , 'Santa Cruz' : 87 , 'Shasta' : 89  , 'Sierra' : 91  , 
 'Siskiyou' : 93 , 'Solano' : 95  , 'Sonoma' : 97  , 'Stanislaus' : 99 , 'Sutter' : 101 , 'Tehama' : 103 , 
 'Trinity' : 105 , 'Tulare' : 107 , 'Tuolumne' : 109 , 'Ventura ' : 111 , 'Yolo' : 113 , 'Yuba' : 115 }

# preprocessing  (transaction_description)


In [39]:

tr_descriptions = df['transaction_description']

snowball = nltk.stem.SnowballStemmer('english')

temp = []

for transaction in tr_descriptions:
    transaction = transaction.lower()                                            # changing to lower case
    transaction = "".join(l for l in transaction if l not in string.punctuation) # removing puntuation
    transaction = ''.join([l for l in transaction if not l.isdigit()])           # removing numbers
    words = [snowball.stem(word) for word in transaction.split()                 # stemming the words
             if word not in stopwords.words('english')]                          # remvoving stopwords
    temp.append(words)

# flatten the list
sent = []
for row in temp:
    sequ = ''
    for word in row:
        sequ = sequ + ' ' + word
    sent.append(sequ)
    
tr_descriptions = sent

### change descriptions to numeric vectors

In [ ]:
county_codes = df.recipient_location_county_code

count_vect = CountVectorizer() #ngram or not? ngram_range = (1,2)
BoW = count_vect.fit_transform(tr_descriptions)

tfidf_transformer = TfidfTransformer()
tfidf = tfidf_transformer.fit_transform(BoW)

tfidf

<207883x31906 sparse matrix of type '<class 'numpy.float64'>'
	with 1203525 stored elements in Compressed Sparse Row format>

# targetting class imbalance

In [102]:
unique, counts = np.unique(county_codes, return_counts=True)
print(sorted(dict(zip(unique, counts)).items(), key=lambda kv: kv[1]))




[(51, 1), (3, 3), (43, 5), (91, 32), (5, 52), (105, 55), (9, 62), (69, 63), (115, 71), (109, 112), (49, 113), (101, 115), (63, 120), (103, 121), (11, 137), (55, 138), (31, 163), (57, 232), (99, 259), (25, 266), (95, 271), (53, 272), (21, 279), (17, 290), (77, 292), (35, 335), (39, 339), (79, 378), (47, 387), (15, 421), (29, 421), (33, 421), (107, 481), (61, 519), (27, 521), (7, 566), (93, 747), (111, 910), (19, 915), (89, 928), (97, 1122), (23, 1166), (41, 1189), (45, 1446), (87, 1491), (83, 1757), (71, 2160), (65, 2486), (13, 2855), (81, 5275), (67, 6821), (113, 8246), (59, 10883), (1, 15410), (85, 20943), (75, 25681), (73, 42330), (37, 44818)]


#### Three states with very few observations
- Mono (51): 1
- Alpine (3): 3
- Mariposa (43): 5

# implementation of the ML model

In [21]:
# dont't know what this does exactly
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.model_selection import train_test_split, cross_val_score, KFold

#models
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier

from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report

## using cross validation score

In [27]:
models = [
    LinearSVC(class_weight='balanced'),
    #MultinomialNB(),
    RandomForestClassifier(),
    #LogisticRegression(random_state=0)
]

CV = KFold(n_splits = 3, shuffle = True, random_state = 0)
cv_df = pd.DataFrame(index = range(3 * len(models)))


# Which model do we choose?
entries = []

for model in models:
    model_name = model.__class__.__name__
    accuracies = cross_val_score(model, tfidf, county_codes, scoring = 'accuracy', cv = CV)
    
    for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))
    
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

# we measure the mean accuracy for each model.
cv_df.groupby('model_name').accuracy.mean()

/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
 

model_name
LinearSVC                 0.591225
RandomForestClassifier    0.759332
Name: accuracy, dtype: float64

In [28]:
accuracies

array([0.76018067, 0.75840224, 0.75941238])

In [38]:
tfidf

<207892x31906 sparse matrix of type '<class 'numpy.float64'>'
	with 1203569 stored elements in Compressed Sparse Row format>

## using train_test_split

In [43]:
model = RandomForestClassifier()
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(tfidf, county_codes,
                                                                                 df.index, test_size=0.3,
                                                                                 shuffle = True,
                                                                                 stratify = county_codes,
                                                                                 random_state=0)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)


#print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


              precision    recall  f1-score   support

           1       0.82      0.73      0.77      4623
           5       0.00      0.00      0.00        16
           7       0.25      0.06      0.10       170
           9       0.00      0.00      0.00        19
          11       1.00      0.12      0.22        41
          13       0.68      0.53      0.60       857
          15       0.73      0.36      0.48       126
          17       0.86      0.28      0.42        87
          19       0.74      0.27      0.40       274
          21       1.00      0.07      0.13        84
          23       0.60      0.14      0.23       350
          25       0.88      0.17      0.29        80
          27       0.44      0.08      0.13       156
          29       0.66      0.33      0.44       126
          31       0.67      0.16      0.26        49
          33       0.94      0.13      0.22       126
          35       0.80      0.04      0.08       100
          37       0.68    

/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
# confusion matrix
confusion_mat = confusion_matrix(y_test, y_pred)
df_confmat = pd.DataFrame(confusion_mat)
sns.heatmap(df_confmat, annot=True)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()

pd.Series(np.diag(df_confmat)).sum()













## using cross validation (understanding it)

In [12]:
model = RandomForestClassifier()
scores = cross_val_score(model, tfidf, county_codes, cv=3)

/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [13]:
scores


array([0.4604576 , 0.56559447, 0.36240022])

## adding total_obligation


In [244]:
from scipy.sparse import coo_matrix, hstack

In [247]:

featureSetAll = hstack((tfidf, df.total_obligation.to_frame()))

In [15]:
model = RandomForestClassifier()
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(featureSetAll, county_codes,
                                                                                 df.index, test_size=0.3,
                                                                                 random_state=60)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)


print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred))

NameError: name 'featureSetAll' is not defined

# understanding data imbalance

In [154]:
unique, counts = np.unique(county_codes, return_counts=True)
print(sorted(dict(zip(unique, counts)).items(), key=lambda kv: kv[0]))



[(1, 15410), (5, 52), (7, 566), (9, 62), (11, 137), (13, 2855), (15, 421), (17, 290), (19, 915), (21, 279), (23, 1166), (25, 266), (27, 521), (29, 421), (31, 163), (33, 421), (35, 335), (37, 44818), (39, 339), (41, 1189), (45, 1446), (47, 387), (49, 113), (53, 272), (55, 138), (57, 232), (59, 10883), (61, 519), (63, 120), (65, 2486), (67, 6821), (69, 63), (71, 2160), (73, 42330), (75, 25681), (77, 292), (79, 378), (81, 5275), (83, 1757), (85, 20943), (87, 1491), (89, 928), (91, 32), (93, 747), (95, 271), (97, 1122), (99, 259), (101, 115), (103, 121), (105, 55), (107, 481), (109, 112), (111, 910), (113, 8246), (115, 71)]


In [153]:
unique, counts = np.unique(y_test, return_counts=True)
print(sorted(dict(zip(unique, counts)).items(), key=lambda kv: kv[1]))



[(91, 6), (5, 14), (105, 16), (69, 17), (9, 19), (115, 27), (63, 29), (101, 29), (49, 33), (109, 36), (103, 42), (11, 45), (31, 51), (55, 52), (57, 66), (95, 73), (25, 80), (17, 84), (77, 84), (53, 90), (39, 92), (21, 94), (99, 96), (35, 99), (47, 103), (29, 107), (15, 112), (79, 117), (107, 124), (33, 125), (7, 153), (27, 161), (61, 162), (93, 229), (111, 260), (89, 262), (19, 294), (97, 320), (23, 335), (41, 365), (45, 425), (87, 434), (83, 479), (71, 665), (65, 759), (13, 862), (81, 1559), (67, 2012), (113, 2472), (59, 3317), (1, 4679), (85, 6163), (75, 7701), (73, 12842), (37, 13493)]


In [30]:
temp = classification_report(y_test, y_pred, output_dict = True)
class_report = pd.DataFrame(temp).transpose()

class_report[class_report.recall < 0.7]

/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,f1-score,precision,recall,support
3,0.000000,0.000000,0.000000,2.0
5,0.000000,0.000000,0.000000,20.0
7,0.088608,0.736842,0.047138,297.0
9,0.066667,1.000000,0.034483,29.0
11,0.352941,1.000000,0.214286,70.0
13,0.558216,0.671968,0.477401,1416.0
15,0.416216,0.472393,0.371981,207.0
17,0.391304,0.782609,0.260870,138.0
19,0.365733,0.770833,0.239741,463.0
21,0.097222,0.777778,0.051852,135.0


In [ ]:
# What are the classes we want to improve

classes = {57:50} #class 57

ros = RandomOverSampler(ratio=classes, random_state=0)
x_resampled, y_resampled = ros.fit_resample(tfidf, county_codes)

print(sorted(Counter(y_resampled).items()))

# things to remember


### what about hashing?

If I understand correctly, you essentially have two forms of features for your models. (1) Text data that you have represented as a sparse bag of words and (2) more traditional dense features. If that is the case then there are 3 common approaches:

- Perform dimensionality reduction (such as LSA via TruncatedSVD) on your sparse data to make it dense and combine the features into a single dense matrix to train your model(s).

- Add your few dense features to your sparse matrix using something like scipy's hstack into a single sparse matrix to train your model(s).

- Create a model using only your sparse text data and then combine its predictions (probabilities if it's classification) as a dense feature with your other dense features to create a model (ie: ensembling via stacking). 

If you go this route remember to only use CV predictions as features to train your model otherwise you'll likely overfit quite badly (you can make a quite class to do this all within a single Pipeline if desired).


- Accuracy is not the metric to use when working with an imbalanced dataset. We have seen that it is misleading

Things to look at:



- Confusion Matrix: A breakdown of predictions into a table showing correct predictions (the diagonal) and the types of incorrect predictions made (what classes incorrect predictions were assigned).
- Precision: A measure of a classifiers exactness.
- Recall: A measure of a classifiers completeness
- F1 Score (or F-score): A weighted average of precision and recall.

I would also advice you to take a look at the following:

- Kappa (or Cohen’s kappa): Classification accuracy normalized by the imbalance of the classes in the data.
- ROC Curves: Like precision and recall, accuracy is divided into sensitivity and specificity and models can be chosen based on the balance thr